In [1]:
import pandas as pd
import numpy as np
import nltk
from datasets import Dataset, DatasetDict
from transformers import AutoModel, AutoTokenizer
from typing import Union
from skmultilearn.model_selection import iterative_train_test_split

MODEL_NAME = "neuralmind/bert-base-portuguese-cased"

/home/vitor/.cache/pypoetry/virtualenvs/pln-trabalho-final-9al6VtFs-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/vitor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
file = 'public_data/train/track_a/ptbr.csv'
data = pd.read_csv(file)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226 entries, 0 to 2225
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2226 non-null   object
 1   text      2226 non-null   object
 2   Anger     2226 non-null   int64 
 3   Disgust   2226 non-null   int64 
 4   Fear      2226 non-null   int64 
 5   Joy       2226 non-null   int64 
 6   Sadness   2226 non-null   int64 
 7   Surprise  2226 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 139.2+ KB


In [4]:
data.head()

,id,text,Anger,Disgust,Fear,Joy,Sadness,Surprise
0,ptbr_train_track_a_00001,"minha vó me disse que era frango e eu comi, ti...",0,0,0,0,1,0
1,ptbr_train_track_a_00002,Está e a nossa deputada Benedita linda guerrei...,0,0,0,1,0,0
2,ptbr_train_track_a_00003,só falta as roupas kkkkkkkkkkk,0,0,0,1,0,0
3,ptbr_train_track_a_00004,Eu tmb. Comecei a sair de casa agora (fui pela...,0,0,0,0,1,0
4,ptbr_train_track_a_00005,Peço a Deus que nossos dirigentes tenham realm...,0,0,0,0,0,0


### Pré-processamento dos dados
1. Case folding
2. Remover stop words
3. Remover acentuação e pontuação


In [5]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords[:20]

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele']

In [6]:
data["clean_text"] = data["text"].copy()
# Case folding
data["clean_text"] = data["clean_text"].str.lower()
# Remover stopwords
data["clean_text"] = data["clean_text"].replace({r"\b" + stopword + r"\b": "" for stopword in stopwords}, regex=True)
# Remover acentuação e pontuação
data["clean_text"] = data["clean_text"].str.replace(r"[\.!?\\\-.,]", "", regex=True)
data["clean_text"] = data["clean_text"].str.replace(r"\s+", " ", regex=True)


In [7]:
data["text"], data["clean_text"]

(0       minha vó me disse que era frango e eu comi, ti...
 1       Está e a nossa deputada Benedita linda guerrei...
 2                          só falta as roupas kkkkkkkkkkk
 3       Eu tmb. Comecei a sair de casa agora (fui pela...
 4       Peço a Deus que nossos dirigentes tenham realm...
                               ...                        
 2221              Eu acho que o CAP vai surpreender hein.
 2222    23:59 - Lula sabia de toda a corrupção no seu ...
 2223    O Brasil precisa URGENTE de pessoas sérias e c...
 2224    Sera que só eu acho que ta passando da hora de...
 2225                                     falta só 2 porra
 Name: text, Length: 2226, dtype: object,
 0         vó disse frango comi gosto frango mto inocente 
 1        deputada benedita linda guerreira parabéns ju...
 2                                falta roupas kkkkkkkkkkk
 3        tmb comecei sair casa agora ( primeira vez ci...
 4       peço deus dirigentes realmente iluminação toma...
              

### Adicionar emoção neutra

Para casos que não tem nenhuma emoção na linha, adicionamos uma nova emoção: neutro

In [8]:
BASE_EMOTIONS = ["Anger", "Disgust", "Fear", "Joy", "Sadness", "Surprise"]

data["Neutral"] = 0
no_emotions_mask = data[BASE_EMOTIONS].sum(axis=1) == 0
data.loc[no_emotions_mask, "Neutral"] = 1

In [9]:
EMOTIONS = BASE_EMOTIONS + ["Neutral"]

In [15]:
import numpy as np
from skmultilearn.model_selection import iterative_train_test_split

# Example dataset
X = np.random.rand(100, 1)  # 100 samples, 1 feature
y = np.random.randint(0, 2, size=(100, 6))  # 100 samples, 6 multi-hot encoded labels

# Perform iterative train-test split
X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size=0.2)

# Outputs
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (80, 1), y_train shape: (80, 6)
X_test shape: (20, 1), y_test shape: (20, 6)


### Iterative train test split
Uma vez que o dataset é desbalanceado, precisamos garantir que os dados de treino e teste tenham proporções similares de cada classe. Entretanto, já que o nosso problema é multi classe, utilizar o *train_test_split* do scikit-learn não funciona, uma vez que ele não lida bem com problemas multi classe, pois nesse tipo de problema há muitas combinações de classe possíveis. Sendo assim, utilizamos a função *iterative_train_test_split* que se propõe a resolver esse problema

In [ ]:
def concat_X_y(X: np.array, y: np.array, columns: list[str]) -> pd.DataFrame:
    concatted_np = np.concatenate((X, y), axis=1)
    concatted = pd.DataFrame(concatted_np, columns=columns)
    return concatted


def train_test_val_split(
        data: pd.DataFrame,
        feature_labels: str,
        targets_labels: list[str]
    ) -> Union[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    X = data[feature_labels].to_numpy().reshape(-1, 1)
    y = data[targets_labels].to_numpy()
    
    X_train, y_train, X_test, y_test = iterative_train_test_split(
        X,
        y,
        test_size=0.1,
    )
    X_train, y_train, X_val, y_val = iterative_train_test_split(
        X_train,
        y_train,
        test_size=0.1,
    )


    return X_train, y_train, X_val, y_val, X_test, y_test
train_test_val_split(data, "clean_text", EMOTIONS)


(1804, 1) (1804, 7)
[[' vó disse frango comi gosto frango mto inocente ' 0 0 ... 1 0 0]
 [' deputada benedita linda guerreira parabéns juntos' 0 0 ... 0 0 0]
 [' vivo usar zulive' 0 0 ... 0 0 1]
 ...
 [' brasil precisa urgente pessoas sérias comprometidas bem população brasileira vamos concentrar assuntos importantes firulas poder grana dentro próprio partido nao sabedoria pra conduzir partido político discernimento sabedoria nao condicoes conduzir algo maior'
  1 0 ... 0 0 0]
 ['sera acho ta passando hora impeachment pra tirar cara' 1 0 ... 0 0 0]
 ['falta 2 porra' 0 0 ... 0 0 0]]


(array([[' vó disse frango comi gosto frango mto inocente '],
        [' deputada benedita linda guerreira parabéns juntos'],
        [' vivo usar zulive'],
        ...,
        [' brasil precisa urgente pessoas sérias comprometidas bem população brasileira vamos concentrar assuntos importantes firulas poder grana dentro próprio partido nao sabedoria pra conduzir partido político discernimento sabedoria nao condicoes conduzir algo maior'],
        ['sera acho ta passando hora impeachment pra tirar cara'],
        ['falta 2 porra']], dtype=object),
 array([[0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([['peço deus dirigentes realmente iluminação tomar decisões certas tempo certo deus abençõe presidente nação fica paz'],
        ['perdi tudo pal itú kkkkk realmente sexo apenas centímetros'],
        ['qndo pensa iremos morre

In [ ]:
train[EMOTIONS].sum()/len(train), val[EMOTIONS].sum()/len(val), test[EMOTIONS].sum()/len(test)

(Anger       0.325194
 Disgust     0.031632
 Fear        0.049390
 Joy         0.260266
 Sadness     0.148724
 Surprise    0.067148
 Neutral     0.283574
 dtype: float64,
 Anger       0.323383
 Disgust     0.029851
 Fear        0.034826
 Joy         0.278607
 Sadness     0.144279
 Surprise    0.079602
 Neutral     0.268657
 dtype: float64,
 Anger       0.300448
 Disgust     0.053812
 Fear        0.058296
 Joy         0.251121
 Sadness     0.112108
 Surprise    0.071749
 Neutral     0.300448
 dtype: float64)

### Preparação para a estrutura do Huggingface

In [ ]:
def create_dataset_dict(train: pd.DataFrame, val: pd.DataFrame, test: pd.DataFrame) -> DatasetDict:
    train_dataset = Dataset.from_pandas(train)
    val_dataset = Dataset.from_pandas(val)
    test_dataset = Dataset.from_pandas(test)

    return DatasetDict({
        'train': train_dataset,
        'validation': val_dataset,
        'test': test_dataset
    })

dataset = create_dataset_dict(train, val, test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

### Finetuning do modelo

In [ ]:
model = AutoModel.from_pretrained(MODEL_NAME)